# Task 1

Demographic parity = $\frac{0.65}{0.5} = 1.3$ \\
Equal opportunity = $\frac{0.75}{0.5} = 1.5$ \\
Positive predictive rate parity = $\frac{\frac{60}{65}}{0.5} = 1.84$ \\
Negative predictive rate parity = $\frac{\frac{20}{35}}{0.5} = 1.14$ \\

#Task 2

## Raport

I train the model on Adult Income Dataset from Kagge.


Description: \\
An individual’s annual income results from various factors. Intuitively, it is influenced by the individual’s education level, age, gender, occupation, and etc.

Fields
The dataset contains 16 columns
Target filed: Income
-- The income is divide into two classes: <=50K and >50K
Number of attributes: 14
-- These are the demographics and other features to describe a person

### 2.

I trained shallow XGBoost model (`max_depth=2`) with the following results

| Statistical parity | Equal opportunity | Predictive parity |
| --- | --- | --- |
| 0.294 | 0.804 | 1.059 |

Predictive parity is way beyond acceptable interval, therefore model cannot be consider fair.

### 3.

Next I trained deeper model of the same architecture (`max_depth=10`).
![](https://drive.google.com/uc?export=view&id=1pwsZhdeBBl9ZjnGte8OIES0FXyOMvVV8)

In this case, allowing model to train more complex features decreases its bias. All metrics are improved.

### 4.

I am applying reweighting mitigation technique.

![](https://drive.google.com/uc?export=view&id=15vcph7j4riutQzCjOaKwyz-KnQOcRadF)

As one can see, this method significantly improved Predictive equality and Statistical parity ratio. However, there is a trade-off - metrics of Equal opportunity and Predictive parity increased.

### 5.


| Model | Recall | Precision | f1 | Accuracy | AUC |
| --- | --- | --- | --- | --- | --- |
| XGB | .554 | .784 | .649 | .852 | .911 |
| XGB_deep | .635 | .773 | .697 | .863 | .924 |
| XGB_reweight | .538 | .786 | .639 | .849 | .906 |

`XGB_deep` is better than `XGB` in almost every metric. As expected, after applying bias mitigation technique - reweighting - `XGB_reweight` is the worst model in terms of performance (apart from Precision). However, drop in performance is not so large, while fairness gained by this method seems worth the trouble.

# Appendix

## Import packages

In [1]:
!pip install dalex &>/dev/null
!pip install lime &>/dev/null
!pip install shap &>/dev/null

In [2]:
import numpy as np
import pandas as pd

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, accuracy_score, confusion_matrix,\
  ConfusionMatrixDisplay, f1_score, recall_score, precision_score
from sklearn.preprocessing import StandardScaler

import xgboost as xgb
import dalex as dx
import lime
import shap

SEED = 42

## Load data

In [3]:
!gdown 10BKanAcpv09lOPD6QsVtOHivuJEwc92N

Downloading...
From: https://drive.google.com/uc?id=10BKanAcpv09lOPD6QsVtOHivuJEwc92N
To: /content/adult.csv
100% 5.33M/5.33M [00:00<00:00, 56.1MB/s]


In [4]:
df_raw = pd.read_csv('adult.csv')
df_raw.head()

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K
4,18,?,103497,Some-college,10,Never-married,?,Own-child,White,Female,0,0,30,United-States,<=50K


## Data preprocessing

In [5]:
df_raw.info() # only int and float

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48842 entries, 0 to 48841
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   age              48842 non-null  int64 
 1   workclass        48842 non-null  object
 2   fnlwgt           48842 non-null  int64 
 3   education        48842 non-null  object
 4   educational-num  48842 non-null  int64 
 5   marital-status   48842 non-null  object
 6   occupation       48842 non-null  object
 7   relationship     48842 non-null  object
 8   race             48842 non-null  object
 9   gender           48842 non-null  object
 10  capital-gain     48842 non-null  int64 
 11  capital-loss     48842 non-null  int64 
 12  hours-per-week   48842 non-null  int64 
 13  native-country   48842 non-null  object
 14  income           48842 non-null  object
dtypes: int64(6), object(9)
memory usage: 5.6+ MB


In [6]:
print(df_raw.shape) # 48842 observations, 15 variables (including one output class)
df_raw.describe()

(48842, 15)


,age,fnlwgt,educational-num,capital-gain,capital-loss,hours-per-week
count,48842.000000,4.884200e+04,48842.000000,48842.000000,48842.000000,48842.000000
mean,38.643585,1.896641e+05,10.078089,1079.067626,87.502314,40.422382
std,13.710510,1.056040e+05,2.570973,7452.019058,403.004552,12.391444
min,17.000000,1.228500e+04,1.000000,0.000000,0.000000,1.000000
25%,28.000000,1.175505e+05,9.000000,0.000000,0.000000,40.000000
50%,37.000000,1.781445e+05,10.000000,0.000000,0.000000,40.000000
75%,48.000000,2.376420e+05,12.000000,0.000000,0.000000,45.000000
max,90.000000,1.490400e+06,16.000000,99999.000000,4356.000000,99.000000


Based on the data and documentation from kaggle we should apply one-hot encoding transformation to the following variables: `cp`, `restecg`, `slp` and `thall`.

In [7]:
df = df_raw[(df_raw != '?').all(axis=1)].drop(columns='fnlwgt')
df.loc[:, df.dtypes == 'object'] = df.select_dtypes(['object']).apply(lambda x: x.astype('category'))

X = df.drop(columns='income')
# convert gender to binary only because the `max_cat_to_onehot` parameter in XGBoost is yet to be working properly..
X = pd.get_dummies(X, columns=X.columns[X.dtypes == 'category'], drop_first=True) 
y = df.income == '>50K'
y.name = 'high_income'

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=SEED, stratify=y)

With stratify argument we can assure than the proportion of values in the sample produced will be the same as the proportion of values in `y`.

Also, I decided to standardize features by removing the mean and scaling to unit variance. We don't want a feature that has a variance that is orders of magnitude larger than others.

For now I will use just train/test split, in the future I can extend it with train/val/test split if needed.

## Train a XGBoost model

In [8]:
BST_model = xgb.XGBClassifier(random_state=SEED, max_depth=2).fit(X_train, y_train)

## Evaluate model on some examples

In [9]:
pf_xgboost_classifier_default = lambda m, d: m.predict_proba(d)[:, 1]
BST_explainer = dx.Explainer(BST_model, X_test, y_test, predict_function=pf_xgboost_classifier_default, label="XGB")

Preparation of a new explainer is initiated

  -> data              : 13567 rows 95 cols
  -> target variable   : Parameter 'y' was a pandas.Series. Converted to a numpy.ndarray.
  -> target variable   : 13567 values
  -> model_class       : xgboost.sklearn.XGBClassifier (default)
  -> label             : XGB
  -> predict function  : <function <lambda> at 0x7fe190f73af0> will be used
  -> predict function  : Accepts only pandas.DataFrame, numpy.ndarray causes problems.
  -> predicted values  : min = 0.00219, mean = 0.246, max = 0.995
  -> model type        : classification will be used (default)
  -> residual function : difference between y and yhat (default)
  -> residuals         : min = -0.961, mean = 0.00189, max = 0.996
  -> model_info        : package xgboost

A new explainer has been created!


Check performance

In [10]:
BST_explainer.model_performance()

,recall,precision,f1,accuracy,auc
XGB,0.553837,0.784,0.64912,0.851625,0.910541


Let's set gender as a protected attribute

In [12]:
protected_variable = X_test.gender_Male.apply(lambda x: "male" if x else "female")
privileged_group = "male"

fobject = BST_explainer.model_fairness(
    protected=protected_variable,
    privileged=privileged_group
)

## Bias

In [13]:
fobject.fairness_check()

Bias detected in 2 metrics: FPR, STP

Conclusion: your model is not fair because 2 or more criteria exceeded acceptable limits set by epsilon.

Ratios of metrics, based on 'male'. Parameter 'epsilon' was set to 0.8 and therefore metrics should be within (0.8, 1.25)
             TPR       ACC       PPV       FPR      STP
female  0.803853  1.130967  1.059126  0.178082  0.29386


In [14]:
fobject.plot()

## Deeper model

In [18]:
BST_model_deep = xgb.XGBClassifier(random_state=SEED, max_depth=10, min_split_loss=2).fit(X_train, y_train)
BST_explainer_deep = dx.Explainer(BST_model_deep, X_test, y_test, predict_function=pf_xgboost_classifier_default, label="XGB_deep")

Preparation of a new explainer is initiated

  -> data              : 13567 rows 95 cols
  -> target variable   : Parameter 'y' was a pandas.Series. Converted to a numpy.ndarray.
  -> target variable   : 13567 values
  -> model_class       : xgboost.sklearn.XGBClassifier (default)
  -> label             : XGB_deep
  -> predict function  : <function <lambda> at 0x7fe190f73af0> will be used
  -> predict function  : Accepts only pandas.DataFrame, numpy.ndarray causes problems.
  -> predicted values  : min = 0.000212, mean = 0.247, max = 0.999
  -> model type        : classification will be used (default)
  -> residual function : difference between y and yhat (default)
  -> residuals         : min = -0.984, mean = 0.00119, max = 1.0
  -> model_info        : package xgboost

A new explainer has been created!


In [19]:
BST_explainer_deep.model_performance()

,recall,precision,f1,accuracy,auc
XGB_deep,0.634741,0.772629,0.69693,0.863197,0.924088


In [20]:
fobject_deep = BST_explainer_deep.model_fairness(
    protected=protected_variable,
    privileged=privileged_group
)
fobject_deep.fairness_check()

Bias detected in 2 metrics: FPR, STP

Conclusion: your model is not fair because 2 or more criteria exceeded acceptable limits set by epsilon.

Ratios of metrics, based on 'male'. Parameter 'epsilon' was set to 0.8 and therefore metrics should be within (0.8, 1.25)
             TPR       ACC       PPV       FPR       STP
female  0.865741  1.114183  0.988372  0.270588  0.338462


In [21]:
fobject.plot(fobject_deep, show=False).\
    update_layout(autosize=False, width=800, height=450, legend=dict(yanchor="top", y=0.99, xanchor="right", x=0.99))

## Bias mitigation

In [28]:
from dalex.fairness import reweight
from copy import copy

protected_variable_train = X_train.gender_Male.apply(lambda x: "male" if x else "female")
privileged_group = "male"

# reweight
sample_weight = reweight(
    protected_variable_train, 
    y_train, 
    verbose=False
)
BST_model_reweight = copy(BST_model)
BST_model_reweight.fit(X_train, y_train, sample_weight=sample_weight)
explainer_reweight = dx.Explainer(
    BST_model_reweight, 
    X_test, 
    y_test, 
    label='XGBClassifier with Reweight mitigation',
    verbose=False
)
fobject_reweight = explainer_reweight.model_fairness(
    protected_variable, 
    privileged_group
)

In [34]:
explainer_reweight.model_performance()

,recall,precision,f1,accuracy,auc
XGBClassifier with Reweight mitigation,0.537775,0.785745,0.638531,0.849119,0.905726


In [29]:
fobject_reweight.fairness_check()

Bias detected in 2 metrics: TPR, STP

Conclusion: your model is not fair because 2 or more criteria exceeded acceptable limits set by epsilon.

Ratios of metrics, based on 'male'. Parameter 'epsilon' was set to 0.8 and therefore metrics should be within (0.8, 1.25)
             TPR    ACC       PPV  FPR  STP
female  1.253385  1.125  0.801944  0.9  0.6


In [33]:
fobject.plot([fobject_deep, fobject_reweight], show=False).\
    update_layout(autosize=False, width=800, height=450, legend=dict(yanchor="top", y=0.99, xanchor="left", x=0.99))

---
---